Пятнашки


In [8]:
from copy import deepcopy

class PuzzleGame:
    def __init__(self, initial_state):
        self.initial_state = initial_state
        self.matrix = []

        self.MOVE = {
            'UP'    :   (-1, 0),
            'DOWN'  :   (1, 0), 
            'LEFT'  :   (0, -1),
            'RIGHT' :   (0, 1)
            }

    def get_matrix(self):
        matrix = ''
        for row in self.matrix:
            matrix += str(row) + '\n'

        return matrix

    def __str__(self):
        return self.get_matrix()

    def check_moves(self, prev_move):
        if prev_move == None:
            return tuple(self.MOVE.keys())

        if prev_move == 'UP':
            return ('UP', 'LEFT', 'RIGHT')

        if prev_move == 'DOWN':
            return ('DOWN', 'LEFT', 'RIGHT')

        if prev_move == 'LEFT':
            return ('UP', 'DOWN', 'LEFT')

        if prev_move == 'RIGHT':
            return ('UP', 'DOWN', 'RIGHT')

    def move_allowed(self, place, move):
        row, col = self.new_place(place, move)
        if row > 0 and row < 5 and col > 0 and col < 5:
            return True

        else:
            return False

    def new_place(self, place, move):
        return (place[0]+move[0], place[1]+move[1])

    def apply_move(self, place, new_place):
        o_r, o_c = place
        n_r, n_c = new_place

        new_matrix = deepcopy(self.matrix)

        new_matrix[n_r-1][n_c-1] = new_matrix[o_r-1][o_c-1]
        new_matrix[o_r-1][o_c-1] = self.matrix[n_r-1][n_c-1]

        return new_matrix

    def value_replaced(self, new_place):
        n_r, n_c = new_place
        return self.matrix[n_r-1][n_c-1]

    def distance(self, prev_place, new_place):
        x1, y1 = prev_place
        x2, y2 = new_place
        d = abs(x2 - x1) + abs(y2 - y1)

        return d

    def count_dissimilar(self, matrix):
        dissmilar = 0
        count = 1
        for row in matrix:
            for col in row:
                if col and col is not count:
                    dissmilar += 1

                count += 1

        return dissmilar

    def generate_matrix(self, initial_state):
        return [initial_state[i:i+4] for i in range(0, len(initial_state), 4)]

    def generate_numbers(self, grid_size):
        return [i for i in range(1, grid_size)] + [None]

    def is_solvable(self):
        self.place = self.get_block_place(block=None, matrix=self.matrix)
        result = self.sum_less_i() + self.x(block_place=self.place)

        return (result % 2 == 0)

    def get_block_place(self, block, matrix): # возвращает координаты None
        for row in range(0, 4):
            for col in range(0, 4):
                if (matrix[row][col] is block):
                    return (row+1, col+1)

    def sum_less_i(self):
        sumission = 0
        for num in range(0, 16):
            if self.initial_state[num] is None:
                sumission += (16 - (num + 1))
                continue
            
            for current in range(num + 1, 16):
                if (self.initial_state[current] is None):
                    continue

                if (self.initial_state[num] > self.initial_state[current]):
                    sumission += 1

        return sumission

    def x(self, block_place):
        row, col = block_place
        # block is at even row and even col
        # or odd row and odd col 
        if (row + col) % 2 == 0:
            return 0

        else:
            return 1


In [9]:
import random
from time import sleep


initial_state = [5, 1, 7, 3, 9, 2, 11, 4, 13, 6, 15, 8, None, 10, 14, 12]
# initial_state = [1, 2, 3, 4, 5, 6, None, 8, 9, 10, 7, 11, 13, 14, 15, 12]

# initial_state = [12, 1, 10, 2, 7, 11, 4, 14, 5, None, 9, 15, 8, 13, 6, 3]
# initial_state = [2, 5, 13, 12, 1, None, 3, 15, 9, 7, 14, 6, 10, 11, 8, 4]
# initial_state = [5, 2, 4, 8, 10, None, 3, 14, 13, 6, 11, 12, 1, 15, 9, 7]
# initial_state = [11, 4, 12, 2, 5, 10, 3, 15, 14, 1, 6, 7, None, 9, 8, 13]
# initial_state = [5, 8, 7, 11, 1, 6, 12, 2, 9, None, 13, 10, 14, 3, 4, 15]

game = PuzzleGame(initial_state)

game.matrix = game.generate_matrix(initial_state)
print(game)

if (game.is_solvable()):
    random.seed(0)
    print("Goal is reachable!")
    dissimilars = game.count_dissimilar(game.matrix)
    place = game.get_block_place(block=None, matrix=game.matrix)
    prev_move = None
    level = 0

    ## Initiate game
    while(dissimilars > 0):
        level += 1

        moves = game.check_moves(prev_move)
        print("Moves at level {}: {}".format(level, moves))
        dissimilar = {}
        for move in moves:
            if (game.move_allowed(place, game.MOVE[move])):
                new = game.new_place(place, game.MOVE[move])
                new_matrix = game.apply_move(place, new)

                dissimilars = game.count_dissimilar(new_matrix)
                dissimilar[move] = dissimilars

        print("Dissimilars : {}".format(dissimilar))

        dissimilars = min(dissimilar.values())

        dissimilar_moves = list(dissimilar.keys())
        random.shuffle(dissimilar_moves)

        for move in dissimilar_moves:
            if dissimilar[move] == dissimilars:
                prev_move = move

        # dissimilars -= level

        print("Selected move : {}".format(prev_move))

        new_place = game.new_place(place, game.MOVE[prev_move])
        game.matrix = game.apply_move(place, new_place)
        place = new_place
        print(game)
        sleep(0.2)

    print("Total path cost = {}".format(level - 1))

else:
    print("Goal is unreachable!")

[5, 1, 7, 3]
[9, 2, 11, 4]
[13, 6, 15, 8]
[None, 10, 14, 12]

Goal is reachable!
Moves at level 1: ('UP', 'DOWN', 'LEFT', 'RIGHT')
Dissimilars : {'UP': 14, 'RIGHT': 15}
Selected move : UP
[5, 1, 7, 3]
[9, 2, 11, 4]
[None, 6, 15, 8]
[13, 10, 14, 12]

Moves at level 2: ('UP', 'LEFT', 'RIGHT')
Dissimilars : {'UP': 13, 'RIGHT': 14}
Selected move : UP
[5, 1, 7, 3]
[None, 2, 11, 4]
[9, 6, 15, 8]
[13, 10, 14, 12]

Moves at level 3: ('UP', 'LEFT', 'RIGHT')
Dissimilars : {'UP': 12, 'RIGHT': 13}
Selected move : UP
[None, 1, 7, 3]
[5, 2, 11, 4]
[9, 6, 15, 8]
[13, 10, 14, 12]

Moves at level 4: ('UP', 'LEFT', 'RIGHT')
Dissimilars : {'RIGHT': 11}
Selected move : RIGHT
[1, None, 7, 3]
[5, 2, 11, 4]
[9, 6, 15, 8]
[13, 10, 14, 12]

Moves at level 5: ('UP', 'DOWN', 'RIGHT')
Dissimilars : {'DOWN': 10, 'RIGHT': 11}
Selected move : DOWN
[1, 2, 7, 3]
[5, None, 11, 4]
[9, 6, 15, 8]
[13, 10, 14, 12]

Moves at level 6: ('DOWN', 'LEFT', 'RIGHT')
Dissimilars : {'DOWN': 9, 'LEFT': 11, 'RIGHT': 10}
Selected move 